In [7]:
# 导入库和加载数据
import torch    # 负责张量,CPU/GPU运算,自动求导(autograd),基础数学运算
import torch.nn as nn   # 负责定义神经网络结构(Linear,Conv,RNN,......),以及损失函数，管理模型参数等
import torch.optim as optim     # 梯度算完了，那w怎么变?它管的是用什么策略更新参数
from torchvision import datasets,transforms    # 专门为视觉任务准备的库,
from torch.utils.data import DataLoader     # 负责划分batch,shuffle以及拼接batch


# 读取图像数据
transform = transforms.Compose([
    transforms.ToTensor(),  # 将加载的图像数据从PIL图像或NumPy ndarray转换成PyTorch的张量（Tensor）
                            # 具体来说，它把图片的格式从 (Height, Width, Channel) 变为 (Channel, Height, Width)，符合PyTorch对输入数据的通道优先格式要求。
                            # 同时，将像素值从整数范围 [0, 255] 归一化到 [0.0, 1.0] 浮点数区间。
])
"""
理由：
PyTorch的神经网络模块接受的数据必须是Tensor格式，且通常是浮点数，用于方便梯度计算和数值稳定。
网络训练时使用归一化后的浮点数输入，保证数值尺度统一，有助于反向传播和优化过程稳定。
"""


# 加载cifar_10数据集的训练集
train_dataset = datasets.CIFAR10(
    root = "./data",    # 记录文件路径(这里会直接在当前文件夹下存放数据集)
    train = True,   # 加载cifar_10数据集的训练集
    download = True,    # 如果本地没找到数据，则自动下载
    transform = transform   # 这个transform就是上面的那个transform，是一个函数对象
)


# 加载cifar_10数据集的训练集
test_dataset = datasets.CIFAR10(
    root = './data',
    train = False,  # 让 Dataset 使用“测试集那一部分数据”
    download = True,
    transform=transform
)
"""
CIFAR10是一个继承自Dataset的数据集对象
这个对象里面通常包含：
文件路径
数据索引
标签列表
transform（函数对象）
一些元信息 
"""



# 创建用于训练的数据加载器
train_loader = DataLoader(
    train_dataset,
# 这是数据源。就是我们上一步创建的那个包含了50000张训练图片和标签的 CIFAR10 数据集对象。
# DataLoader 会从这里面取数据。
    
    batch_size = 64,
# 意思是“批次大小为64”

# """
# DataLoader 不会一次性把50000张图片全部加载到内存里（这会撑爆内存），而是每次只取出64张图片及其对应的标签，
# 打包成一个批次。
# 为什么这么做?
# 内存效率：防止内存溢出。
# 训练效率：GPU并行处理64张图片比一张一张处理要快得多。
# 优化稳定性：用一个批次（64张图片）的平均梯度来更新模型参数，比只用一张图片（梯度随机性大）或全部图片（计算成本太高）
# 要更稳定和高效。这就是所谓的小批量梯度下降 (Mini-batch Gradient Descent)。
# """

    shuffle = True
# 意思是“打乱数据”


# """
# 在每个训练周期（epoch）开始时，DataLoader 会随机打乱整个 train_dataset 的顺序，然后再按顺序取出批次。
# 为什么这么做？（非常重要！）
# 打破数据顺序的依赖：如果数据是按类别排序的（比如先是1000张猫，然后1000张狗……），模型在训练初期会连续看到同一
# 种类的样本，导致梯度更新方向出现偏差，学得不好。
# 提高模型泛化能力：每次训练看到的批次组合都不同，这增加了训练的随机性，能有效防止模型“背诵”训练数据的顺序，
# 从而学习到更通用的特征。这就像你学习时，不按课本章节顺序，而是随机抽查知识点，这样记得更牢。
# """
)


# 创建用于测试的数据加载器
test_loader = DataLoader(
    test_dataset,
    batch_size = 64,
    shuffle = False
# 意思是不打乱数据
# """
# DataLoader 会始终按照固定的顺序来加载测试数据。
# 为什么这么做？
# 评估的确定性：测试的目的是评估模型性能，我们希望每次评估的结果都是一致、可复现的。如果打乱了顺序，
# 虽然最终的准确率不变，但每次运行的中间过程会不同，不利于调试和比较。
# 没有必要：测试时我们不更新模型参数，只是前向传播计算结果，所以数据顺序对模型的最终评估指标（如准确率）没有影响。
# 打乱顺序没有收益，反而失去了结果的可复现性。

# """

)




In [8]:
# 查看你导入的数据图片(在实际搭建模型中不需要这一步)
img,label = train_dataset[0]
print(type(img))



<class 'torch.Tensor'>
